In [22]:
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

In [23]:
# retrieve data from external source; create pandas DataFrame

mcas = pd.read_csv('https://github.com/jp213/EL/raw/master/sample-mcas.csv')
proc_sample = pd.read_csv('https://github.com/jp213/EL/raw/master/mcas-processed.csv')

In [24]:
# DataFrame overview (shape, columns, Dtypes, nulls, etc.)

mcas.info(); print(); proc_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Columns: 238 entries, district to summarize
dtypes: int64(20), object(218)
memory usage: 4.9+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCESID             2 non-null      int64  
 1   StudentTestID      2 non-null      int64  
 2   StudentLocalID     2 non-null      int64  
 3   StudentGradeLevel  2 non-null      int64  
 4   TestDate           2 non-null      object 
 5   TestName           2 non-null      object 
 6   TestTypeName       2 non-null      object 
 7   TestSubjectName    2 non-null      object 
 8   TestGradeLevel     2 non-null      int64  
 9   Score1Label        2 non-null      object 
 10  Score1Type         2 non-null      object 
 11  Score1Value        2 non-null      object 
 12  Score2Label        2 non-null      object 
 13  Sc

In [25]:
# assign only relevant columns to new df; new df overview

mcase = mcas[['district', 'sasid', 'stugrade', 'eperf2', 'escaleds', 'ecpi',
              'mperf2', 'mscaleds', 'mcpi', 'sperf2', 'sscaleds', 'scpi']].copy()
mcase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  2671 non-null   object
 1   sasid     2671 non-null   int64 
 2   stugrade  2671 non-null   object
 3   eperf2    2671 non-null   object
 4   escaleds  2671 non-null   object
 5   ecpi      2671 non-null   object
 6   mperf2    2671 non-null   object
 7   mscaleds  2671 non-null   object
 8   mcpi      2671 non-null   object
 9   sperf2    2671 non-null   object
 10  sscaleds  2671 non-null   object
 11  scpi      2671 non-null   object
dtypes: int64(1), object(11)
memory usage: 250.5+ KB


In [26]:
# create/populate column for unique district IDs to align with specs

unique_districts = list(mcase.district.unique())
mcase.insert(1, 'NCESID', 0)

for i in range(len(unique_districts)):
    for n in range(len(mcase)):
        if mcase.district[n] == unique_districts[i]:
            mcase.at[n, 'NCESID'] = 373737 + i

In [27]:
# insert columns to align with specs

mcase.insert(3, 'StudentTestID', mcase.sasid)
mcase.insert(4, 'StudentLocalID', None)
mcase.insert(5, 'StudentGradeLevel', mcase.stugrade)
mcase.insert(6, 'TestDate', None)
mcase.insert(7, 'TestName', 'MCAS')
mcase.insert(8, 'TestTypeName', None)
mcase.insert(9, 'TestSubjectName', None)
mcase.insert(10, 'TestGradeLevel', mcase.stugrade)
mcase.insert(11, 'Score1Label', 'Performance Level')
mcase.insert(12, 'Score1Type', 'Level')
mcase.insert(13, 'Score1Value', None)
mcase.insert(14, 'Score2Label', 'Scaled Score')
mcase.insert(15, 'Score2Type', 'Scale')
mcase.insert(16, 'Score2Value', np.nan)
mcase.insert(17, 'Score3Label', 'CPI')
mcase.insert(18, 'Score3Type', 'Scale')
mcase.insert(19, 'Score3Value', np.nan)

In [28]:
# checking for duplicate student IDs

mcase.StudentTestID.value_counts()

1840155    2
1603583    1
1619332    1
1140098    1
1555839    1
          ..
1659356    1
1129142    1
1043124    1
1187782    1
1445888    1
Name: StudentTestID, Length: 2670, dtype: int64

In [29]:
# note and display instances of duplicates

print('There are two students with the same SASID: a 7th grade female with DOB 9/10/99, a 10th grade male with DOB 5/28/96.')
print('\nPlease ask the district to look into this.')
mcas[['school', 'sasid', 'grade', 'gender', 'dob']].loc[mcas['sasid'] == 1840155]

There are two students with the same SASID: a 7th grade female with DOB 9/10/99, a 10th grade male with DOB 5/28/96.

Please ask the district to look into this.


,school,sasid,grade,gender,dob
1363,404,1840155,7,F,9/10/99
2450,505,1840155,10,M,5/28/96


In [30]:
# inspecting unique values (objects) for StudentGradeLevel

mcase.StudentGradeLevel.unique()

array(['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'SP'],
      dtype=object)

In [31]:
# inspecting unique values (objects) for performance level scores

print(mcase.eperf2.sort_values().unique()); print()
print(mcase.mperf2.sort_values().unique()); print()
print(mcase.sperf2.sort_values().unique())

[' ' 'A' 'F' 'NI' 'P' 'W']

[' ' 'A' 'F' 'NI' 'P' 'W']

[' ' 'A' 'F' 'NI' 'P' 'W']


In [32]:
# Mapping performance level scores to align with integration specs
# code for 'P+' included for future cases

mcase['epl'] = None
mcase['mpl'] = None
mcase['spl'] = None

for i in range(len(mcase)):
    if mcase.eperf2[i] == 'A':
        mcase.at[i, 'epl'] = '5 - A'
    elif mcase.eperf2[i] == 'F':
        mcase.at[i, 'epl'] = '1 - F'
    elif mcase.eperf2[i] == 'NI':
        mcase.at[i, 'epl'] = '3 - NI'
    elif mcase.eperf2[i] == 'P':
        mcase.at[i, 'epl'] = '4 - P'
    elif mcase.eperf2[i] == 'P+':
        mcase.at[i, 'epl'] = '6 - P+'
    elif mcase.eperf2[i] == 'W':
        mcase.at[i, 'epl'] = '2 - W'
    else:
        pass

for i in range(len(mcase)):
    if mcase.mperf2[i] == 'A':
        mcase.at[i, 'mpl'] = '5 - A'
    elif mcase.mperf2[i] == 'F':
        mcase.at[i, 'mpl'] = '1 - F'
    elif mcase.mperf2[i] == 'NI':
        mcase.at[i, 'mpl'] = '3 - NI'
    elif mcase.mperf2[i] == 'P':
        mcase.at[i, 'mpl'] = '4 - P'
    elif mcase.mperf2[i] == 'P+':
        mcase.at[i, 'mpl'] = '6 - P+'
    elif mcase.mperf2[i] == 'W':
        mcase.at[i, 'mpl'] = '2 - W'
    else:
        pass

for i in range(len(mcase)):
    if mcase.sperf2[i] == 'A':
        mcase.at[i, 'spl'] = '5 - A'
    elif mcase.sperf2[i] == 'F':
        mcase.at[i, 'spl'] = '1 - F'
    elif mcase.sperf2[i] == 'NI':
        mcase.at[i, 'spl'] = '3 - NI'
    elif mcase.sperf2[i] == 'P':
        mcase.at[i, 'spl'] = '4 - P'
    elif mcase.sperf2[i] == 'P+':
        mcase.at[i, 'spl'] = '6 - P+'
    elif mcase.sperf2[i] == 'W':
        mcase.at[i, 'spl'] = '2 - W'
    else:
        pass

In [33]:
# inspecting unique values (objects) for scaled scores

print(mcase.escaleds.sort_values().unique()); print()
print(mcase.mscaleds.sort_values().unique()); print()
print(mcase.sscaleds.sort_values().unique())

[' ' '208' '210' '212' '214' '216' '218' '220' '222' '224' '226' '228'
 '230' '232' '234' '236' '238' '240' '242' '244' '246' '248' '250' '252'
 '254' '256' '258' '260' '262' '264' '266' '268' '270' '272' '274' '276'
 '278' '280']

[' ' '202' '204' '206' '208' '210' '212' '214' '216' '218' '220' '222'
 '224' '226' '228' '230' '232' '234' '236' '238' '240' '242' '244' '246'
 '248' '250' '252' '254' '256' '258' '260' '262' '264' '266' '268' '270'
 '272' '274' '276' '278' '280']

[' ' '206' '208' '210' '212' '214' '216' '218' '220' '222' '224' '226'
 '228' '230' '232' '234' '236' '238' '240' '242' '244' '246' '248' '250'
 '252' '254' '256' '258' '260' '262' '264' '266' '268' '270' '272' '274'
 '276' '278' '280']


In [34]:
# strip whitespace, convert to numeric/NaN to allow calculations

mcase['esc'] = pd.to_numeric(mcase.escaleds.str.strip())
mcase['msc'] = pd.to_numeric(mcase.mscaleds.str.strip())
mcase['ssc'] = pd.to_numeric(mcase.sscaleds.str.strip())

In [35]:
# inspecting unique values (objects) for CPI scores

print(mcase.ecpi.sort_values().unique()); print()
print(mcase.mcpi.sort_values().unique()); print()
print(mcase.scpi.sort_values().unique())

[' ' '0' '100' '25' '50' '75']

[' ' '0' '100' '25' '50' '75']

[' ' '0' '100' '25' '50' '75']


In [36]:
# strip whitespace, convert to numeric/NaN to allow calculations

mcase['ei'] = pd.to_numeric(mcase.ecpi.str.strip())
mcase['mi'] = pd.to_numeric(mcase.mcpi.str.strip())
mcase['si'] = pd.to_numeric(mcase.scpi.str.strip())

In [37]:
# drop columns no longer needed; assign remaining columns to new DataFrame

drop_list = ['district', 'sasid', 'stugrade', 'eperf2', 'mperf2', 'sperf2',
             'escaleds', 'mscaleds', 'sscaleds', 'ecpi', 'mcpi', 'scpi']
mcasi = mcase.drop(drop_list, 1)

In [38]:
# overview of new DataFrame; view first 5 rows

mcasi.info(); mcasi.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCESID             2671 non-null   int64  
 1   StudentTestID      2671 non-null   int64  
 2   StudentLocalID     0 non-null      object 
 3   StudentGradeLevel  2671 non-null   object 
 4   TestDate           0 non-null      object 
 5   TestName           2671 non-null   object 
 6   TestTypeName       0 non-null      object 
 7   TestSubjectName    0 non-null      object 
 8   TestGradeLevel     2671 non-null   object 
 9   Score1Label        2671 non-null   object 
 10  Score1Type         2671 non-null   object 
 11  Score1Value        0 non-null      object 
 12  Score2Label        2671 non-null   object 
 13  Score2Type         2671 non-null   object 
 14  Score2Value        0 non-null      float64
 15  Score3Label        2671 non-null   object 
 16  Score3Type         2671 

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,epl,mpl,spl,esc,msc,ssc,ei,mi,si
0,373737,1408905,None,3,None,MCAS,None,None,3,Performance Level,Level,None,Scaled Score,Scale,NaN,CPI,Scale,NaN,2 - W,2 - W,None,216.0,210.0,NaN,25.0,25.0,NaN
1,373737,1020285,None,3,None,MCAS,None,None,3,Performance Level,Level,None,Scaled Score,Scale,NaN,CPI,Scale,NaN,3 - NI,4 - P,None,238.0,244.0,NaN,75.0,100.0,NaN
2,373737,1681155,None,3,None,MCAS,None,None,3,Performance Level,Level,None,Scaled Score,Scale,NaN,CPI,Scale,NaN,3 - NI,3 - NI,None,220.0,236.0,NaN,50.0,75.0,NaN
3,373737,1987801,None,3,None,MCAS,None,None,3,Performance Level,Level,None,Scaled Score,Scale,NaN,CPI,Scale,NaN,4 - P,4 - P,None,252.0,246.0,NaN,100.0,100.0,NaN
4,373737,1351732,None,3,None,MCAS,None,None,3,Performance Level,Level,None,Scaled Score,Scale,NaN,CPI,Scale,NaN,4 - P,3 - NI,None,246.0,228.0,NaN,100.0,50.0,NaN


In [39]:
%%time
rebuilt = pd.DataFrame()

for i in range(len(mcasi)):
    
    if type(mcasi['epl'].iloc[i]) != str\
    and math.isnan(mcasi['esc'].iloc[i])\
    and math.isnan(mcasi['ei'].iloc[i])\
    and type(mcasi['mpl'].iloc[i]) != str\
    and math.isnan(mcasi['msc'].iloc[i])\
    and math.isnan(mcasi['mi'].iloc[i])\
    and type(mcasi['spl'].iloc[i]) != str\
    and math.isnan(mcasi['ssc'].iloc[i])\
    and math.isnan(mcasi['si'].iloc[i]):
    
        # designate rowo as original row
        rowo = mcasi.iloc[[i]]
        rebuilt = rebuilt.append(rowo)
    
    else:
        
        # if any ELA fields have non-nulls
        if type(mcasi['epl'].iloc[i]) == str\
        or mcasi['esc'].iloc[i] >= 0\
        or mcasi['ei'].iloc[i] >= 0:
        
            # designate rowe as ELA row
            rowe = mcasi.iloc[[i]]
            rowe = rowe.copy()
            rowe.TestDate = 'April 1'
            rowe.TestTypeName = 'MCAS ELA'
            rowe.TestSubjectName = 'ELA'
            rowe.Score1Value = rowe.epl
            rowe.Score2Value = rowe.esc
            rowe.Score3Value = rowe.ei
            rebuilt = rebuilt.append(rowe)
        
        # if any Math fields have non-nulls
        if type(mcasi['mpl'].iloc[i]) == str\
        or mcasi['msc'].iloc[i] >= 0\
        or mcasi['mi'].iloc[i] >= 0:
        
            # designate rowm as Math row
            rowm = mcasi.iloc[[i]]
            rowm = rowm.copy()
            rowm.TestDate = 'May 1'
            rowm.TestTypeName = 'MCAS Math'
            rowm.TestSubjectName = 'Math'
            rowm.Score1Value = rowm.mpl
            rowm.Score2Value = rowm.msc
            rowm.Score3Value = rowm.mi
            rebuilt = rebuilt.append(rowm)
    
        # if any Science fields have non-nulls
        if type(mcasi['spl'].iloc[i]) == str\
        or mcasi['ssc'].iloc[i] >= 0\
        or mcasi['si'].iloc[i] >= 0:
        
            # designate rows as Science row
            rows = mcasi.iloc[[i]]
            rows = rows.copy()
            rows.TestDate = 'June 1'
            rows.TestTypeName = 'MCAS Science'
            rows.TestSubjectName = 'Science'
            rows.Score1Value = rows.spl
            rows.Score2Value = rows.ssc
            rows.Score3Value = rows.si
            rebuilt = rebuilt.append(rows)

Wall time: 20.5 s


In [40]:
# drop columns no longer needed; assign remaining columns to new DataFrame

drop_list2 = ['epl', 'mpl', 'spl', 'esc', 'msc', 'ssc', 'ei', 'mi', 'si']
proc_mcas = rebuilt.drop(drop_list2, 1)

In [41]:
proc_mcas.info()
proc_mcas.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6231 entries, 0 to 2670
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCESID             6231 non-null   int64  
 1   StudentTestID      6231 non-null   int64  
 2   StudentLocalID     0 non-null      object 
 3   StudentGradeLevel  6231 non-null   object 
 4   TestDate           6198 non-null   object 
 5   TestName           6231 non-null   object 
 6   TestTypeName       6198 non-null   object 
 7   TestSubjectName    6198 non-null   object 
 8   TestGradeLevel     6231 non-null   object 
 9   Score1Label        6231 non-null   object 
 10  Score1Type         6231 non-null   object 
 11  Score1Value        6075 non-null   object 
 12  Score2Label        6231 non-null   object 
 13  Score2Type         6231 non-null   object 
 14  Score2Value        6052 non-null   float64
 15  Score3Label        6231 non-null   object 
 16  Score3Type         6231 

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value
0,373737,1408905,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,2 - W,Scaled Score,Scale,216.0,CPI,Scale,25.0
0,373737,1408905,None,3,May 1,MCAS,MCAS Math,Math,3,Performance Level,Level,2 - W,Scaled Score,Scale,210.0,CPI,Scale,25.0
1,373737,1020285,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,3 - NI,Scaled Score,Scale,238.0,CPI,Scale,75.0
1,373737,1020285,None,3,May 1,MCAS,MCAS Math,Math,3,Performance Level,Level,4 - P,Scaled Score,Scale,244.0,CPI,Scale,100.0
2,373737,1681155,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,3 - NI,Scaled Score,Scale,220.0,CPI,Scale,50.0


In [42]:
proc_mcas.reset_index(drop=True, inplace=True)
proc_mcas.info()
proc_mcas.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6231 entries, 0 to 6230
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NCESID             6231 non-null   int64  
 1   StudentTestID      6231 non-null   int64  
 2   StudentLocalID     0 non-null      object 
 3   StudentGradeLevel  6231 non-null   object 
 4   TestDate           6198 non-null   object 
 5   TestName           6231 non-null   object 
 6   TestTypeName       6198 non-null   object 
 7   TestSubjectName    6198 non-null   object 
 8   TestGradeLevel     6231 non-null   object 
 9   Score1Label        6231 non-null   object 
 10  Score1Type         6231 non-null   object 
 11  Score1Value        6075 non-null   object 
 12  Score2Label        6231 non-null   object 
 13  Score2Type         6231 non-null   object 
 14  Score2Value        6052 non-null   float64
 15  Score3Label        6231 non-null   object 
 16  Score3Type         6231 

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value
0,373737,1408905,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,2 - W,Scaled Score,Scale,216.0,CPI,Scale,25.0
1,373737,1408905,None,3,May 1,MCAS,MCAS Math,Math,3,Performance Level,Level,2 - W,Scaled Score,Scale,210.0,CPI,Scale,25.0
2,373737,1020285,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,3 - NI,Scaled Score,Scale,238.0,CPI,Scale,75.0
3,373737,1020285,None,3,May 1,MCAS,MCAS Math,Math,3,Performance Level,Level,4 - P,Scaled Score,Scale,244.0,CPI,Scale,100.0
4,373737,1681155,None,3,April 1,MCAS,MCAS ELA,ELA,3,Performance Level,Level,3 - NI,Scaled Score,Scale,220.0,CPI,Scale,50.0


In [43]:
proc_mcas.to_csv('proc_mcas.csv')